#FIRFilter
 

*   Mykola Liashuha
*   Jose Gonzalez Lopez



#Introduction
In this work we will use the audio from the previous exercise to create a FIR delay effect. First, we will determine the impulse response of the filter. Next, the frequency response of the filter will be calculated. After that,the delay effect will be applied to the audio. Last, we will conclude the notebook by wraping up all the concepts explored in the exercise.


#Imports

In [1]:
import numpy as np #Numpy for calculations
import sklearn #Use this library to normalize
import math #For calculations
from scipy.io import wavfile #Read wav files
from matplotlib import pyplot as plt #Plots
from scipy.fft import fft, ifft,fftfreq #Fourier transform

#Auxiliar Functions

In [2]:
#Code here

#The Filter
Our main goal is to implement the following FIR delay effect using the formula:

$s[t]=e[t]+αe[t−D]$

where α is the attenuation factor, $D$ is the time delay, $s$ is the output signal and $e$ the input. Our first objective will be to get the impulse response of the filter ($h[t]$). Let's assume that $α=0.1$ and $D=1$. Our formula now will be:

$s[t]=e[t]+0.1 \cdot e[t−1]$

We will also assume that the signal is causal and stable. Having that:

$s[0]=e[0]$

$s[1]=e[1]+0.1 \cdot e[0]$

$s[2]=e[2]+0.1 \cdot e[1]$

$s[3]=e[3]+0.1 \cdot e[2]$

And so on, then the impulse respond will be:

$h[t]=\delta[t] + \cdot 0.1 \delta[t-1]$

Where $\delta[t]$ is the dirac function

In [3]:
samplerate, data = wavfile.read('string_1.wav') #Read the data stream and get samplerate
ALPHA=0.1 #Attenuation Factor
DELAY=1 #Time delay

#References
[How to calculate the impulse response](https://dsp.stackexchange.com/questions/53512/how-to-find-the-impulse-response-of-a-filter-given-the-difference-equation)